In [ ]:
pip install transformers accelerate datasets peft bitsandbytes safetensors deepspeed


# CPU

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

# Choose the GPT-Neo model (1.3B or other smaller variants)
BASE_MODEL = "EleutherAI/gpt-neo-1.3B"  # Smaller GPT-Neo model

# Load tokenizer with force download option
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, use_auth_token=True, force_download=True)

# Load the GPT-Neo model without CUDA (only CPU)
model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    device_map="cpu",  # Use CPU only
    torch_dtype=torch.float32,  # Use full precision (float32) for CPU
    force_download=True  # Force fresh download
)

# Prepare the model for LoRA training
model = prepare_model_for_kbit_training(model)


tokenizer_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

tokenizer_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/5.31G [00:00<?, ?B/s]

# GPU

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from bitsandbytes import BitsAndBytesConfig

# Choose the GPT-Neo model (1.3B or other smaller variants)
BASE_MODEL = "EleutherAI/gpt-neo-1.3B"  # Smaller GPT-Neo model

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({"pad_token": "<PAD>"})

# Use BitsAndBytesConfig for 4-bit quantization
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,      # Enable 4-bit quantization
    bnb_4bit_compute_dtype=torch.float16,  # Use float16 for efficiency
    llm_int8_enable_fp32_cpu_offload=True  # Enable offloading to CPU in FP32
)

# Load the GPT-Neo model with quantization config
model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    quantization_config=quantization_config,
    device_map="auto",  # Automatically distribute model to GPU/CPU
    torch_dtype=torch.float16  # Use float16 precision
)

# Prepare model for LoRA training
model = prepare_model_for_kbit_training(model)


The 8-bit optimizer is not available on your device, only available on CUDA for now.


ImportError: cannot import name 'BitsAndBytesConfig' from 'bitsandbytes' (d:\Personal Project_certificate work Space(Projects )\salesforce project\salesforce-integration\venv\Lib\site-packages\bitsandbytes\__init__.py)

In [13]:
from datasets import load_dataset

# Load your dataset from the train.jsonl file
ds = load_dataset("json", data_files="train.jsonl")["train"]


In [14]:
ds

Dataset({
    features: ['input', 'output'],
    num_rows: 10
})

In [ ]:
from transformers import AutoTokenizer

# Load the tokenizer for GPT-Neo (or your chosen model)
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-1.3B")

# Ensure the tokenizer has a padding token
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': tokenizer.eos_token})  # Set pad_token to eos_token

# Tokenization function
def tokenize_fn(batch):
    # Tokenize the input text (raw notes) and output text (formatted notes)
    model_inputs = tokenizer(batch["input"], truncation=True, padding=True, max_length=512)
    
    # For causal language modeling, labels should be the same as inputs (shifted)
    labels = tokenizer(batch["output"], truncation=True, padding=True, max_length=512).input_ids
    
    # Align the labels with inputs for causal language modeling
    model_inputs["labels"] = labels
    return model_inputs

# Apply tokenization and padding, and remove unnecessary columns
tok_ds = ds.map(tokenize_fn, batched=True, remove_columns=["input", "output"])

# Check the dataset after tokenization
print(tok_ds[0])  # To inspect how the tokenized data looks 


{'input_ids': [12832, 1326, 11421, 784, 327, 10468, 3409, 3469, 25, 220, 198, 15926, 233, 5905, 1850, 1459, 6884, 26, 1262, 10655, 8740, 44, 13, 220, 198, 48860, 287, 8868, 10107, 1366, 5726, 416, 2319, 7225, 220, 198, 33234, 1431, 4466, 3800, 290, 2551, 15264, 357, 48, 19, 737, 50256, 50256, 50256, 50256], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0], 'labels': [22093, 25, 198, 12, 8444, 29569, 1459, 6884, 26, 1262, 10655, 8740, 44, 379, 4013, 1326, 11421, 13, 198, 12, 327, 10468, 25, 3409, 3469, 13, 198, 198, 9218, 15627, 11045, 25, 198, 12, 12033, 276, 287, 8868, 10107, 1366, 5726, 416, 2319, 7225, 198, 198, 12502, 17230, 25, 198, 12, 5521, 319, 3557, 803, 8740, 44, 7767, 284, 4646, 10107, 5128, 13, 198, 12, 978, 570, 351, 4466, 3800, 290, 2551, 15264, 357, 48, 19, 737, 198, 198, 10019, 32144, 25, 198, 12, 19281, 1061, 12, 929, 3249, 329, 1195, 19, 5114, 13,

In [16]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

# Define the LoRA configuration
peft_config = LoraConfig(
    r=4,                         # Low rank for LoRA
    lora_alpha=16,               # Scaling factor for LoRA
    target_modules=["q_proj", "v_proj"],  # Apply LoRA to attention layers
    lora_dropout=0.1,            # Dropout rate for LoRA
    task_type="CAUSAL_LM"        # Causal language modeling task type
)

# Prepare the GPT-Neo model with LoRA
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("EleutherAI/gpt-neo-1.3B", device_map="auto")

# Apply LoRA to the model
model = get_peft_model(model, peft_config)

# Use .to_empty() for devices to avoid meta tensor errors
model = model.to_empty(device="cpu")  # Move model to CPU or CUDA if available


Some parameters are on the meta device because they were offloaded to the disk and cpu.


In [17]:
from transformers import TrainingArguments

# Define the training arguments
training_args = TrainingArguments(
    output_dir="./gpt_neo_lora",             # Directory to save the model
    per_device_train_batch_size=1,           # Use smaller batch size on CPU
    per_device_eval_batch_size=1,            # Batch size for evaluation
    gradient_accumulation_steps=4,           # Accumulate gradients to simulate larger batch size
    num_train_epochs=1,                      # 1 epoch for demonstration
    learning_rate=2e-4,                      # Learning rate
    fp16=False,                              # Disable fp16 since we're using CPU
    logging_steps=5,                         # How often to log during training
    save_strategy="no"                       # Do not save checkpoints (can be changed)
)


In [ ]:
# Manually split the dataset into train and test (90% train, 10% test)
train_test_split = tok_ds.train_test_split(test_size=0.1)

# Now you have `train` and `test` splits
train_dataset = train_test_split["train"]
eval_dataset = train_test_split["test"]


In [20]:
from transformers import Trainer, DataCollatorForSeq2Seq

# Data collator for padding and batching
collator = DataCollatorForSeq2Seq(tokenizer, pad_to_multiple_of=8, return_tensors="pt")

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,   # Use the training split
    eval_dataset=eval_dataset,     # Use the test split
    data_collator=collator
)



In [12]:
# Start the training process
trainer.train()


d:\Personal Project_certificate work Space(Projects )\salesforce project\salesforce-integration\venv\Lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


ValueError: Expected input batch_size (56) to match target batch_size (112).

In [ ]:
# Save the fine-tuned model and tokenizer
model.save_pretrained("./mistral_lora")
tokenizer.save_pretrained("./mistral_lora")


In [ ]:
from peft import PeftModel

# Example raw notes
prompt = """Format into Salesforce notes:

RetailAxis – Merchandising Lead Sarah Gold: 
Struggling with inventory sync. Wants cloud migration plan. Will send inventory reports.

Formatted:"""

inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

# Load base model (before fine-tuning)
base_model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, load_in_4bit=True, device_map="auto")
base_output = base_model.generate(**inputs, max_new_tokens=256)
print("BASE (before fine-tuning):\n", tokenizer.decode(base_output[0], skip_special_tokens=True))

# Load LoRA fine-tuned model
lora_model = PeftModel.from_pretrained(base_model, "./mistral_lora")
lora_output = lora_model.generate(**inputs, max_new_tokens=256)
print("LoRA (after fine-tuning):\n", tokenizer.decode(lora_output[0], skip_special_tokens=True))
